# Chapter 9 System Design and Scalability

from Gayle Laakmann McDowell's "Cracking the Coding Interview", 6th ed.

Ron Wu

## 9.1 stock data 

In [1]:
#json or xml are good for storing (easy adding new data, new columns)
#sql databases are efficient for selecting data

'''
going to 
https://developer.yahoo.com/yql/console/?q=show%20tables&env=store://datatables.org/alltableswithkeys

useful tables:

yahoo.finance.quotes
yahoo.finance.historicaldata 
yahoo.finance.balancesheet
yahoo.finance.incomestatement
yahoo.finance.dividendhistory
yahoo.finance.analystestimate
yahoo.finance.keystats 

then write SQL then get the url

View them on the web https://finance.yahoo.com/quote/FB/key-statistics?p=FB
'''
import requests
from pprint import pprint

url = "https://query.yahooapis.com/v1/public/yql?q=select%20*%20from%20\
        yahoo.finance.historicaldata%20where%20symbol%20%3D%20%22SYMBOL%22%20\
        and%20startDate%20%3D%20%22STARTINGDATE%22%20and%20endDate%20%3D%20%22ENDINGDATE\
        %22&format=json&diagnostics=true&env=store%3A%2F%2Fdatatables.org%2Falltableswithkeys&\
        callback="

url=url.replace("SYMBOL","IBM").replace("STARTINGDATE","2016-10-1").replace("ENDINGDATE","2016-10-5") 

response = requests.get(url)
response_json = response.json()

pprint(response_json["query"]["results"]["quote"])

[{u'Adj_Close': u'157.080002',
  u'Close': u'157.080002',
  u'Date': u'2016-10-05',
  u'High': u'157.830002',
  u'Low': u'156.720001',
  u'Open': u'157.070007',
  u'Symbol': u'IBM',
  u'Volume': u'1684500'},
 {u'Adj_Close': u'156.460007',
  u'Close': u'156.460007',
  u'Date': u'2016-10-04',
  u'High': u'158.529999',
  u'Low': u'155.820007',
  u'Open': u'157.669998',
  u'Symbol': u'IBM',
  u'Volume': u'2884100'},
 {u'Adj_Close': u'157.610001',
  u'Close': u'157.610001',
  u'Date': u'2016-10-03',
  u'High': u'158.369995',
  u'Low': u'157.020004',
  u'Open': u'158.059998',
  u'Symbol': u'IBM',
  u'Volume': u'2227800'}]


## 9.2 social network

In [2]:
#store as graph, and the nodes are userID, userID hashed pointing to 
#user information (including connections) stored on different machines 

#to find a shortest paths between two nodes (A, B), do BFS on both nodes. 
#If the hash tables says some of A's friends live on machine 'C',
#then wait to see some many B's connections live on 'C' too, then
#and go to 'C' in a batch because jumping between machines have high latency

#The hash table itself may be too big for one machine. Splitting hash table to different
#machines, keep hash collisions on the same machine, so for a given user, first
#get the hash value, so we know which machine contains the segment of hash table that 
#the user belongs to, then from the hash table get the machine that user information lives 


## 9.3 web crawler

In [3]:
from IPython.display import HTML
HTML('<iframe src=http://quotes.wsj.com/FB/historical-prices width=700 height=500></iframe>')

In [4]:
#In 9.1 I showed how to use yahoo's api
#but for most websites to get data, we need to crawl

import requests
from bs4 import BeautifulSoup

# say we want to crawler http://quotes.wsj.com/FB/historical-prices

url = 'http://quotes.wsj.com/FB/historical-prices'
response = requests.get(url)
bs_text = BeautifulSoup(response.text, "html.parser")

In [5]:
import pyperclip 
pyperclip.copy(bs_text)
print(bs_text.prettify()[90250:90250+400])

e_nav">
         <span>
          <input class="datePicker" id="selectDateFrom" type="text" value="07/09/2016"/>
         </span>
         <span>
          to
         </span>
         <span>
          <input class="datePicker" id="selectDateTo" type="text" value="10/07/2016"/>
         </span>
         <span>
          <input id="datPickerButton" type="button" value="go"/>
         </span>
      


In [6]:
import pandas as pd

hist_table = []
hisData = bs_text.find("div",{ "id": "historical_data_table"})
for r in hisData.find_all('tr'):
    row = r.find_all('td')
    date =  row[0].text
    open_price =  row[1].text
    high =  row[2].text
    low =  row[3].text
    close_price =  row[4].text
    vol =  row[5].text.replace(' M','')
    hist_table.append([date, open_price, high, low, close_price, vol])

df = pd.DataFrame(hist_table)
df.columns = ['Date','Open','High','Low','Close','Vol (M)']
df.set_index('Date',inplace=True)
df.head(10)

,Open,High,Low,Close,Vol (M)
Date,,,,,
10/06/16,128.43,129.06,128.08,128.74,11.68
10/05/16,128.25,128.80,127.83,128.47,12.39
10/04/16,129.17,129.2765,127.5499,128.19,14.31
10/03/16,128.38,129.09,127.80,128.77,13.16
09/30/16,128.03,128.59,127.45,128.27,18.40
09/29/16,129.18,129.29,127.55,128.09,14.53
09/28/16,129.21,129.47,128.40,129.23,12.05
09/27/16,127.61,129.01,127.43,128.69,15.64
09/26/16,127.37,128.16,126.80,127.31,15.06


## 9.4 duplicate url

In [7]:
#hash 10 billion url, split hash (base on parent url)
#on different machine and keep collision together and check duplication


## 9.5 cache for web search engine

In [8]:
#search engine doesn't use a brutal search for every request. In fact it builds a power cache
#cache is a hash table split across many machines. When the request comes in, it may hit
#randomly any hash server (work balancer), from the hashed key, they know which machine
#has the hashed information, then go to that machine to look for the hash table.
#the hash value will return a point to a chain of linked list, each node of which stores the
#related page. The LL is very effective for updating page ranks and adding or removing pages.

## 9.6 sales rank

In [9]:
#If we normalize table fully, we would have to have 2 table. 
#One to store sales quantities indexed by product id, and the other is category tables 
#store produces under each category. Then there will be a lots of joins.

#If joins are expensive (because it may cross many machines) but storages are cheap
#we will designate a few machines for each category. When a product is purchased, 
#we log it to all categories this product belongs to. So at the end of day,
#we will do a simple sort for each category

## 9.7 personal financial manager

In [10]:
#what does it do?
#1. pull the banking information: spending, income,
#2. analyze it, catogorize spending, email alert spending over certain percetage of income
#3. user input spending/saving goals.
#4. compare user spending habits across similar users 

## 9.8 pastebin

In [11]:
#hash url to a new message 
#storage url to registered users
#purging message when it expires 